In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from langchain.schema import HumanMessage

In [10]:
from langchain_groq import ChatGroq


In [11]:
import chromadb

from pydantic import BaseModel
from typing import Optional, Callable

import uuid
import json
import os
import types
import inspect
import sys

In [ ]:
API_KEY = ''

In [15]:
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name='bug-reports')
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0, max_tokens=4000,api_key=API_KEY)

UniqueConstraintError: Collection bug-reports already exists

In [14]:
class State(BaseModel):
    function: Callable
    function_string: str
    arguments: list
    error: bool
    error_description: str = ''
    new_function_string: str = ''
    bug_report: str = ''
    memory_search_results: list = []
    memory_ids_to_update: list = []

In [16]:
def code_execution_node(state: State):
    ''' Run Arbitrary Code '''
    try:
        print('\nRunning Arbitrary Function')
        print('--------------------------\n')
        result = state.function(*state.arguments)
        print('\n✅ Arbitrary Function Ran Without Error')
        print(f'Result: {result}')
        print('---------------------------------------\n')
    except Exception as e:
        print(f'❌ Function Raised an Error: {e}')
        state.error = True
        state.error_description = str(e)
    return state


def code_update_node(state: State):
    ''' Update Arbitratry Code '''
    prompt = ChatPromptTemplate.from_template(
        'You are tasked with fixing a Python function that raised an error.'
        'Function: {function_string}'
        'Error: {error_description}' 
        'You must provide a fix for the present error only.'
        'The bug fix should handle the thrown error case gracefully by returning an error message.'
        'Do not raise an error in your bug fix.'
        'The function must use the exact same name and parameters.'
        'Your response must contain only the function definition with no additional text.'
        'Your response must not contain any additional formatting, such as code delimiters or language declarations.'
    )
    message = HumanMessage(content=prompt.format(function_string=state.function_string, error_description=state.error_description))
    new_function_string = llm.invoke([message]).content.strip()

    print('\n🐛 Buggy Function')
    print('-----------------\n')
    print(state.function_string)
    print('\n🩹 Proposed Bug Fix')
    print('-------------------\n')
    print(new_function_string)
    
    state.new_function_string = new_function_string
    return state


def code_patching_node(state: State):
    ''' Fix Arbitrary Code '''
    try:
        print('\n*******************')
        print('\n❤️‍🩹 Patching code...')
        # Store the new function as a string
        new_code = state.new_function_string
        
        # Create namespace for new function
        namespace = {}
        
        # Execute new code in namespace
        exec(new_code, namespace)
        
        # Get function name dynamically
        func_name = state.function.__name__
        
        # Get the new function using dynamic name
        new_function = namespace[func_name]
        
        # Update state
        state.function = new_function
        state.error = False

        # Test the new function
        result = state.function(*state.arguments)

        print('...patch complete 😬\n')
                
    except Exception as e:
        print(f'...patch failed: {e}')
        print(f'Error details: {str(e)}')

    print('******************\n')
    return state

In [17]:
def bug_report_node(state: State):
    ''' Generate Bug Report '''
    prompt = ChatPromptTemplate.from_template(
        'You are tasked with generating a bug report for a Python function that raised an error.'
        'Function: {function_string}'
        'Error: {error_description}'
        'Your response must be a comprehensive string including only crucial information on the bug report'
    )
    message = HumanMessage(content=prompt.format(function_string=state.function_string, error_description=state.error_description))
    bug_report = llm.invoke([message]).content.strip()

    print('\n📝 Generating Bug Report')
    print('------------------------\n')
    print(bug_report)

    state.bug_report = bug_report
    return state

In [18]:
def bug_report_node(state: State):
    ''' Generate Bug Report '''
    prompt = ChatPromptTemplate.from_template(
        'You are tasked with generating a bug report for a Python function that raised an error.'
        'Function: {function_string}'
        'Error: {error_description}'
        'Your response must be a comprehensive string including only crucial information on the bug report'
    )
    message = HumanMessage(content=prompt.format(function_string=state.function_string, error_description=state.error_description))
    bug_report = llm.invoke([message]).content.strip()

    print('\n📝 Generating Bug Report')
    print('------------------------\n')
    print(bug_report)

    state.bug_report = bug_report
    return state


# Digest the bug report using the same template used when saving bug reports to increase the accuracy and relevance of results when querying the vector database.
def memory_search_node(state: State):
    ''' Find memories relevant to the current bug report '''
    prompt = ChatPromptTemplate.from_template(
        'You are tasked with archiving a bug report for a Python function that raised an error.'
        'Bug Report: {bug_report}.'
        'Your response must be a concise string including only crucial information on the bug report for future reference.'
        'Format: # function_name ## error_description ### error_analysis'
    )
    
    message = HumanMessage(content=prompt.format(
        bug_report=state.bug_report,
    ))
    
    response = llm.invoke([message]).content.strip()

    results = collection.query(query_texts=[response])

    print('\n🔎 Searching bug reports...')
    if results['ids'][0]:
        print(f'...{len(results["ids"][0])} found.\n')
        print(results)
        state.memory_search_results = [{'id':results['ids'][0][index], 'memory':results['documents'][0][index], 'distance':results['distances'][0][index]} for index, id in enumerate(results['ids'][0])]
    else:
        print('...none found.\n')
            
    return state


# Filter the top 30% of results to ensure the relevance of memories being updated.
def memory_filter_node(state: State):
    print('\n🗑️ Filtering bug reports...')
    for memory in state.memory_search_results:
        if memory['distance'] < 0.3:
            state.memory_ids_to_update.append(memory['id'])
        
    if state.memory_ids_to_update:
        print(f'...{len(state.memory_ids_to_update)} selected.\n')
    else:
        print('...none selected.\n')
            
    return state


# Condense the bug report before storing it in the vector database.
def memory_generation_node(state: State):
    ''' Generate relevant memories based on new bug report '''
    prompt = ChatPromptTemplate.from_template(
        'You are tasked with archiving a bug report for a Python function that raised an error.'
        'Bug Report: {bug_report}.'
        'Your response must be a concise string including only crucial information on the bug report for future reference.'
        'Format: # function_name ## error_description ### error_analysis'
    )
    
    message = HumanMessage(content=prompt.format(
        bug_report=state.bug_report,
    ))
    
    response = llm.invoke([message]).content.strip()

    print('\n💾 Saving Bug Report to Memory')
    print('------------------------------\n')
    print(response)

    id = str(uuid.uuid4())
    collection.add(
        ids=[id],
        documents=[response],
    )        
    return state


# Use the prior memory as well as the current bug report to generate an updated version of it.
def memory_modification_node(state: State):
    ''' Modify relevant memories based on new interaction '''
    prompt = ChatPromptTemplate.from_template(
        'Update the following memories based on the new interaction:'
        'Current Bug Report: {bug_report}'
        'Prior Bug Report: {memory_to_update}'
        'Your response must be a concise but cumulative string including only crucial information on the current and prior bug reports for future reference.'
        'Format: # function_name ## error_description ### error_analysis'
    )
    memory_to_update_id = state.memory_ids_to_update.pop(0)
    state.memory_search_results.pop(0)
    results = collection.get(ids=[memory_to_update_id])
    memory_to_update = results['documents'][0]
    message = HumanMessage(content=prompt.format(
        bug_report=state.bug_report,
        memory_to_update=memory_to_update,
    ))
    
    response = llm.invoke([message]).content.strip()
    
    print('\nCurrent Bug Report')
    print('------------------\n')
    print(memory_to_update)
    print('\nWill be Replaced With')
    print('---------------------\n')
    print(response)
    
    collection.update(
        ids=[memory_to_update_id],
        documents=[response],
    )
        
    return state

In [20]:
def error_router(state: State):
    if state.error:
        return 'bug_report_node'
    else:
        return END

def memory_filter_router(state: State):
    if state.memory_search_results:
        return 'memory_filter_node'
    else:
        return 'memory_generation_node'


def memory_generation_router(state: State):
    if state.memory_ids_to_update:
        return 'memory_modification_node'
    else:
        return 'memory_generation_node'


def memory_update_router(state: State):
    if state.memory_ids_to_update:
        return 'memory_modification_node'
    else:
        return 'code_update_node'

In [21]:
builder = StateGraph(State)

# Add nodes to the graph
builder.add_node('code_execution_node', code_execution_node)
builder.add_node('code_update_node', code_update_node)
builder.add_node('code_patching_node', code_patching_node)
builder.add_node('bug_report_node', bug_report_node)
builder.add_node('memory_search_node', memory_search_node)
builder.add_node('memory_filter_node', memory_filter_node)
builder.add_node('memory_modification_node', memory_modification_node)
builder.add_node('memory_generation_node', memory_generation_node)

In [22]:
# Add edges to the graph
builder.set_entry_point('code_execution_node')
builder.add_conditional_edges('code_execution_node', error_router)
builder.add_edge('bug_report_node', 'memory_search_node')
builder.add_conditional_edges('memory_search_node', memory_filter_router)
builder.add_conditional_edges('memory_filter_node', memory_generation_router)
builder.add_edge('memory_generation_node', 'code_update_node')
builder.add_conditional_edges('memory_modification_node', memory_update_router)

builder.add_edge('code_update_node', 'code_patching_node')
builder.add_edge('code_patching_node', 'code_execution_node')

# Compile the graph
graph = builder.compile()

In [ ]:
def execute_self_healing_code_system(function, arguments):

    state = State(
        error=False,
        function=function,
        function_string=inspect.getsource(function),
        arguments=arguments,
    )
    
    return graph.invoke(state)

In [24]:
def divide_two_numbers(a, b):
    return a/b

# Test Cases
print("*******************************")
print("*******************************")
print("** Testing Division Function **")
print("*******************************")
print("*******************************")
execute_self_healing_code_system(divide_two_numbers, [10, 0]);


*******************************
*******************************
** Testing Division Function **
*******************************
*******************************

Running Arbitrary Function
--------------------------

❌ Function Raised an Error: division by zero

📝 Generating Bug Report
------------------------

"Bug Report: Division by Zero Error in divide_two_numbers function. Error Message: division by zero. Function Parameters: a and b. Cause: The function does not handle the case when b is zero. Solution: Add a conditional statement to check if b is zero before performing division, raising a meaningful error or returning a specific value when b is zero."


/Users/sseadmin/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:14<00:00, 5.63MiB/s]



🔎 Searching bug reports...
...none found.


💾 Saving Bug Report to Memory
------------------------------

# divide_two_numbers ## Division by Zero Error ### division by zero when b is zero, missing conditional check for b == 0

🐛 Buggy Function
-----------------

def divide_two_numbers(a, b):
    return a/b


🩹 Proposed Bug Fix
-------------------

def divide_two_numbers(a, b):
    try:
        return a/b
    except ZeroDivisionError:
        return "Error: Division by zero is not allowed"

*******************

❤️‍🩹 Patching code...
...patch complete 😬

******************


Running Arbitrary Function
--------------------------


✅ Arbitrary Function Ran Without Error
Result: Error: Division by zero is not allowed
---------------------------------------



In [25]:
execute_self_healing_code_system(divide_two_numbers, ['a', 0]);


Running Arbitrary Function
--------------------------

❌ Function Raised an Error: unsupported operand type(s) for /: 'str' and 'int'

📝 Generating Bug Report
------------------------

"Bug Report: Division Function Error. Function Name: divide_two_numbers. Error Message: unsupported operand type(s) for /: 'str' and 'int'. Description: The function raised an error when attempting to divide a string by an integer, indicating that the function does not handle non-numeric input types. Reproduction Steps: Call the function with a string and an integer, e.g., divide_two_numbers('a', 2). Expected Result: The function should either perform the division or raise a meaningful error. Actual Result: The function raised a TypeError. Proposed Solution: Add input type checking to ensure both inputs are numbers before attempting division."


Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1



🔎 Searching bug reports...
...1 found.

{'ids': [['9f3210bf-a514-4bd2-b4f5-d8b8257e87cc']], 'embeddings': None, 'documents': [['# divide_two_numbers ## Division by Zero Error ### division by zero when b is zero, missing conditional check for b == 0']], 'uris': None, 'data': None, 'metadatas': [[None]], 'distances': [[0.5651549696922302]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}

🗑️ Filtering bug reports...
...none selected.


💾 Saving Bug Report to Memory
------------------------------

# divide_two_numbers ## Division by non-numeric type ## TypeError due to unsupported operand types for division, requires input type checking to handle non-numeric inputs.

🐛 Buggy Function
-----------------

def divide_two_numbers(a, b):
    return a/b


🩹 Proposed Bug Fix
-------------------

def divide_two_numbers(a, b):
    try:
        return a/b
    except TypeError:
        return "Error: Both inputs must be 

In [26]:
def parse_date(date_string):
    year, month, day = date_string.split('-')
    return {'year': int(year), 'month': int(month), 'day': int(day)}

In [27]:
print("** Testing Date Parsing Function **")
print("***********************************")
print("***********************************")
# Test 1: Invalid format
execute_self_healing_code_system(parse_date, ["2024/01/01"]);

** Testing Date Parsing Function **
***********************************
***********************************

Running Arbitrary Function
--------------------------

❌ Function Raised an Error: not enough values to unpack (expected 3, got 1)

📝 Generating Bug Report
------------------------

"Bug Report: The function 'parse_date' raised an error due to insufficient date string values. The error 'not enough values to unpack (expected 3, got 1)' occurs when the input 'date_string' does not contain the expected '-' separator, resulting in only one value being returned by the 'split' method, instead of the required three values for year, month, and day. To fix this, input validation should be added to ensure the date string is in the correct format (YYYY-MM-DD) before attempting to parse it."


Number of requested results 10 is greater than number of elements in index 2, updating n_results = 2



🔎 Searching bug reports...
...2 found.

{'ids': [['610d33a0-a169-4e07-83ac-0a7dab3e4a98', '9f3210bf-a514-4bd2-b4f5-d8b8257e87cc']], 'embeddings': None, 'documents': [['# divide_two_numbers ## Division by non-numeric type ## TypeError due to unsupported operand types for division, requires input type checking to handle non-numeric inputs.', '# divide_two_numbers ## Division by Zero Error ### division by zero when b is zero, missing conditional check for b == 0']], 'uris': None, 'data': None, 'metadatas': [[None, None]], 'distances': [[1.2629616260528564, 1.5617167949676514]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}

🗑️ Filtering bug reports...
...none selected.


💾 Saving Bug Report to Memory
------------------------------

# parse_date ## not enough values to unpack (expected 3, got 1) ### insufficient date string values due to missing '-' separator, requiring input validation for correct YYYY-MM-D

In [28]:
execute_self_healing_code_system(parse_date, ["abc-def-ghi"]);


Running Arbitrary Function
--------------------------

❌ Function Raised an Error: invalid literal for int() with base 10: 'abc'

📝 Generating Bug Report
------------------------

Bug Report: The function 'parse_date' raised a ValueError due to an invalid literal 'abc' when attempting to convert it to an integer. The error occurred because the input 'date_string' contained non-numeric characters, which the function is not designed to handle. To fix this, input validation should be added to ensure 'date_string' is in the correct format 'YYYY-MM-DD' before attempting to parse it.


Number of requested results 10 is greater than number of elements in index 3, updating n_results = 3



🔎 Searching bug reports...
...3 found.

{'ids': [['efb0ac0d-a36a-4e59-8b2e-01fd8356f1ef', '610d33a0-a169-4e07-83ac-0a7dab3e4a98', '9f3210bf-a514-4bd2-b4f5-d8b8257e87cc']], 'embeddings': None, 'documents': [["# parse_date ## not enough values to unpack (expected 3, got 1) ### insufficient date string values due to missing '-' separator, requiring input validation for correct YYYY-MM-DD format.", '# divide_two_numbers ## Division by non-numeric type ## TypeError due to unsupported operand types for division, requires input type checking to handle non-numeric inputs.', '# divide_two_numbers ## Division by Zero Error ### division by zero when b is zero, missing conditional check for b == 0']], 'uris': None, 'data': None, 'metadatas': [[None, None, None]], 'distances': [[0.40036728978157043, 1.0570366382598877, 1.4807682037353516]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}

🗑️ Filtering bug reports...
..